# 以相似度为训练目标

## 生成训练数据，暂存于本地

In [0]:
!git clone https://github.com/daibiaoxuwu/05wan3

In [0]:
!git clone https://github.com/daibiaoxuwu/05wan3b

In [0]:
import os
import cv2
import numpy as np
path1 = '05wan3'
path2 = '05wan3b/0'

clocks = dict()
x_train0 = []
y_train0 = []
for filename in os.listdir(path2):
    if 'git' in filename: continue
    #print(path1)
    #ftimestr,number = path1.split('_')
    clocks[int(filename.split('_')[0])]=int(filename.split('_')[1][:-4])
    #clocks[int(ftimestr)]=int(number)
keys = clocks.keys()
for filename in os.listdir(path1):
    if 'git' in filename or 'c.jpg' in filename: continue
    ftime = int(filename.split('_')[0])
    value = max(filter(lambda t:t<ftime,keys))
    x_train0.append(cv2.imread(os.path.join(path1,filename), cv2.IMREAD_GRAYSCALE)/255.0)
    y_train0.append(clocks[value])
    
print(len(x_train0),len(y_train0))

In [0]:
import seaborn as sns
sns.countplot(y_train0)
print(y_train0[:100])

In [0]:

from matplotlib import pyplot as plt
lst = os.listdir(path1)
for i in range(5):
  plt.imshow(x_train0[i])
  plt.show()
  print(y_train0[i])
  print(lst[i])

# 转为onehot，分离test和train

# 装载Tensorflow

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

# 安装 TensorFlow
try:
  # Colab only
  %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf

# 输入数据生成器
# 变为5张图片一叠

In [0]:
idxes = []
for i in range(10):idxes.append([])
for idx, val in enumerate(y_train0):
  idxes[val].append(idx)
for i in range(10):
  print(idxes[i][:5])

# 搭建训练数据

## 搭建Overfit数据：只有十叠按序的照片

In [0]:
picnum = 5
import random
def train_generator():
  while True:
    dataout = []
    for i in range(5):dataout.append([])
    ansout = np.zeros((100,))
    for train2 in range(100):
      ans = random.randint(0,9)
      idx2 = random.sample(idxes[ans],5)
      for i in range(5):
        dataout[i].append(x_train0[idx2[i]][:,:,np.newaxis])      
      ansout[train2] = ans
      #dataout.append(data)      
    yield [np.stack(data2) for data2 in dataout],ansout 



In [0]:
train_dataset = tf.data.Dataset.from_generator(train_generator,  (tf.float32, tf.float32), ([tf.TensorShape([5,12,12,1])]*5,tf.TensorShape([])))
train_dataset = train_dataset.batch(32)

In [0]:
tx,ty = next(train_generator())
print(tx[0].shape)
print(ty.shape)


# 搭建模型和训练
现在的网络去掉dropout则可以拟合10张图片，
但是加上dropout就拟合不了。
从7张拟合5张的，则可以在两个epoch（每个epoch抽取60000次）内到80%。

继续训练20张，一个epoch就到60%。

In [0]:
picnum = 5
inputs = [tf.keras.layers.Input(shape=(12,12,1))  for i in range(picnum)]
#inputs = tf.keras.layers.Input(shape=(5,12,12,1))

conv_a1 = tf.keras.layers.Conv2D(32, (3, 3), padding = 'Same',  activation='relu')
conva1 = [conv_a1(inputs[i]) for i in range(picnum)]
conv_b1 = tf.keras.layers.Conv2D(32, (3, 3), padding = 'Same',  activation='relu')
convb1 = [conv_b1(conva1[i]) for i in range(picnum)]

conv_c1 = tf.keras.layers.Conv2D(32, (3, 3), padding = 'Same',  activation='relu')
convc1 = [conv_c1(inputs[i]) for i in range(picnum)]
conv_d1 = tf.keras.layers.Conv2D(32, (3, 3), padding = 'Same',  activation='relu')
convd1 = [conv_d1(convc1[i]) for i in range(picnum)]

convs1 = tf.stack(convb1)
convf1 = [
    tf.math.reduce_max(convs1, axis=[0]),
    tf.math.reduce_mean(convs1, axis=[0]),
    tf.math.reduce_min(convs1, axis=[0]),
]

conv_g1 = tf.keras.layers.Conv2D(32, (3, 3), padding = 'Same',  activation='relu')
convg1 = [conv_g1(convf1[i]) for i in range(3)]

convh1 = [tf.concat(convg1+[convd1[i]],axis=3) for i in range(picnum)]
#------------------------------------------------------------------------------------------
conv_a2 = tf.keras.layers.Conv2D(32, (3, 3), padding = 'Same',  activation='relu')
conva2 = [conv_a2(convh1[i]) for i in range(picnum)]
conv_b2 = tf.keras.layers.Conv2D(32, (3, 3), padding = 'Same',  activation='relu')
convb2 = [conv_b2(conva2[i]) for i in range(picnum)]

conv_c2 = tf.keras.layers.Conv2D(32, (3, 3), padding = 'Same',  activation='relu')
convc2 = [conv_c2(inputs[i]) for i in range(picnum)]
conv_d2 = tf.keras.layers.Conv2D(32, (3, 3), padding = 'Same',  activation='relu')
convd2 = [conv_d2(convc2[i]) for i in range(picnum)]

convs2 = tf.stack(convb2)
convf2 = [
    tf.math.reduce_max(convs2, axis=[0]),
    tf.math.reduce_mean(convs2, axis=[0]),
    tf.math.reduce_min(convs2, axis=[0]),
]

conv_g2 = tf.keras.layers.Conv2D(32, (3, 3), padding = 'Same',  activation='relu')
convg2 = [conv_g2(convf2[i]) for i in range(3)]

convh2 = [tf.concat(convg2+[convd2[i]],axis=3) for i in range(picnum)]
#------------------------------------------------------------------------------------------
conv_a3 = tf.keras.layers.Conv2D(32, (3, 3), padding = 'Same',  activation='relu')
conva3 = [conv_a3(convh2[i]) for i in range(picnum)]
conv_b3 = tf.keras.layers.Conv2D(32, (3, 3), padding = 'Same',  activation='relu')
convb3 = [conv_b3(conva3[i]) for i in range(picnum)]

conv_c3 = tf.keras.layers.Conv2D(32, (3, 3), padding = 'Same',  activation='relu')
convc3 = [conv_c3(inputs[i]) for i in range(picnum)]
conv_d3 = tf.keras.layers.Conv2D(32, (3, 3), padding = 'Same',  activation='relu')
convd3 = [conv_d3(convc3[i]) for i in range(picnum)]

convs3 = tf.stack(convb3)
convf3 = [
    tf.math.reduce_max(convs3, axis=[0]),
    tf.math.reduce_mean(convs3, axis=[0]),
    tf.math.reduce_min(convs3, axis=[0]),
]

conv_g3 = tf.keras.layers.Conv2D(32, (3, 3), padding = 'Same',  activation='relu')
convg3 = [conv_g3(convf3[i]) for i in range(3)]

convh3 = tf.concat(convg3+convd3,axis=3)

convx1 = tf.keras.layers.Conv2D(32, (3, 3), padding = 'Same',  activation='relu')(convh3)
convx2 = tf.keras.layers.Conv2D(32, (3, 3), padding = 'Same',  activation='relu')(convx1)
convx3 = tf.keras.layers.Flatten()(convx2)
convy1 = tf.keras.layers.Dense(128,activation='relu')(convx3)

#keep_prob = tf.placeholder(tf.float32) # dropout (keep probability)

#convy2 = tf.keras.layers.Dropout(1)(convy1)
convy3 = tf.keras.layers.Dense(10)(convy1)

model = tf.keras.Model(inputs=inputs, outputs=convy3)

In [0]:


model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.summary()
model.fit_generator(train_generator(),epochs=100,steps_per_epoch=600)

# 去掉三个dropout时，一个epoch后达到99正确率。下面采用乱序的7张照片中取5张。

In [0]:
import random
def train_generator(slice):
  for train in range(60000):
    ans = random.randint(0,9)
    idx2 = random.sample(idxes[ans][:slice],5)
    data = np.stack([x_train0[ix] for ix in idx2],axis=2)
    data = data[:,:,:,np.newaxis]
    ansy = np.zeros(10)
    ansy[ans] = 1
    yield data,ans

In [0]:
train_dataset = tf.data.Dataset.from_generator(train_generator,  (tf.float32, tf.float32), (tf.TensorShape([12,12,5,1]),tf.TensorShape([])), args=(7,))
train_dataset = train_dataset.batch(32)

In [0]:
model.summary()
model.fit(train_dataset,epochs=100)

# 训练并验证模型：

In [0]:
tx,ty = next(train_generator(5))
for i in range(5):
  plt.imshow(tx[:,:,i])
  plt.show()
print(ty)
print(model.predict(np.array((tx,))))